In [ ]:
!pip install transformers
!huggingface-cli login

In [ ]:
!pip install accelerate
!pip install xformers

## Upload datasets and utils

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
import pandas as pd
import re

In [ ]:
path_prefix = "/content/gdrive/MyDrive/DATASETS/QA/"

In [ ]:
cd /content/gdrive/MyDrive/DATASETS/QA/

In [ ]:
QA_DATASETS = [
     {
        "name": "quarel", #ok
        "sep": False,
        "initial_premise": False,
        "end_premise": False,
        "num_options": 2,
        "FLAN": [2, 4, 13],
        "GPT2": [3, 10, 11],
        "BART": [6, 9, 10]
    }, {
        "name": "quartz-no_knowledge", #ok
        "sep": False,
        "initial_premise": False,
        "end_premise": False,
        "num_options": 2,
        "FLAN": [1, 6, 13],
        "GPT2": [1, 5, 10],
        "BART": [2, 3, 6]
    },{
        "name": "quartz-with_knowledge", #ok
        "sep": False,
        "initial_premise": False,
        "end_premise": False,
        "num_options": 2,
        "FLAN": [1, 2, 5],
        "GPT2": [7, 10, 14],
        "BART": [11, 13, 14]
    },
     {
        "name": "race-middle", #ok
        "sep": True,
        "initial_premise": False,
        "end_premise": True,
        "num_options": 4,
        "FLAN": [3, 6, 7],
        "GPT2": [5, 10, 13],
        "BART": [7, 10, 11]
    }, {
        "name": "sciq", #ok
        "sep": True,
        "initial_premise": False,
        "end_premise": True,
        "num_options": 4,
        "FLAN": [2, 6, 8],
        "GPT2": [5, 8, 14],
        "BART": [9, 12, 14]
    }, {
        "name": "social_i_qa", #ok
        "sep": True,
        "initial_premise": False,
        "end_premise": True,
        "num_options": 3,
        "FLAN": [1, 4, 8],
        "GPT2": [1, 4, 14],
        "BART": [0, 2, 4]
    }, {
        "name": "superglue-copa", #ok
        "sep": False,
        "initial_premise": False,
        "end_premise": False,
        "num_options": 2,
        "FLAN": [2, 5, 8],
        "GPT2": [7, 10, 12],
        "BART": [0, 1, 4]
    },
     {
        "name": "wino_grande", #ok
        "sep": False,
        "initial_premise": False,
        "end_premise": False,
        "num_options": 2,
        "FLAN": [1, 10, 11],
        "GPT2": [9, 11, 13],
        "BART": [5, 9, 12]
    }
  ]

In [ ]:
QA_PROMPTS = [
    "Question: {QUESTION}\nOptions: {OPTIONS}\nDo not give an explanation to the answer you give\n",
    "For the following Question Answering task choose the correct answer between the given options. Question: {QUESTION}. Options: {OPTIONS}",
    "Given the following Question Answering task, choose an answer between the options. Question: {QUESTION}. Options: {OPTIONS}",
    "Give an answer for the following Question Answering task. Question: {QUESTION}. Options: {OPTIONS}",
    "Which is the correct answer among the options? Question: {QUESTION}. Options: {OPTIONS}",
    "Choose the correct answer among the options. Question: {QUESTION}. Options: {OPTIONS}",
    "Given these options {OPTIONS}, answer the following question: {QUESTION}. Answer using only the words present in the options.",
    "Answer the following question:\n\n{QUESTION}\n\nOptions: {OPTIONS}",
    "Answer this question:\n\n{QUESTION}?\nOptions: {OPTIONS}",
    "What is the answer to this question? {QUESTION}\nOptions: {OPTIONS}",
    "What is the answer to the following: {QUESTION}? {OPTIONS}",
    "Provide the answer to: {QUESTION}. {OPTIONS}",
    "Answer the following: {QUESTION}. {OPTIONS}",
    "What is the response to: {QUESTION}? {OPTIONS}",
    "In the following text: {QUESTION}, what is the answer? {OPTIONS}",
    "Given the input {QUESTION}, provide the answer. {OPTIONS}"
]

In [ ]:
def handle_initial_sep(query, separator = "\[SEP\]"):
  qpa = re.split(separator, query) #Vector containing in this order QUESTION, PREMISE and ANSWER.
  return f"{qpa[1]}{qpa[0]}{qpa[2]}"

In [ ]:
def handle_end_sep(query, separator = "\[SEP\]", to_remove=[]):
  if (len(to_remove) != 0):
    for substr in to_remove:
      query = query.replace(substr, '')

  qap = re.split(separator, query)

  return f"{qap[1]}{qap[0]}"

In [ ]:
def import_dataset(dataset_index):
  dataset_info = QA_DATASETS[dataset_index];

  dataset_name = dataset_info["name"]
  print(dataset_name)
  #for seed in DATASET_SEEDS:
  seed = 100 #test_set is equal independently on random seed
  test_set_path = f"{dataset_name}/{dataset_name}_32_{seed}_test.tsv"
  #print(test_set_path)
  df = pd.read_csv(test_set_path , sep="\t", header=None)

  has_sep = dataset_info["sep"]
  has_initial_premise = dataset_info["initial_premise"]
  has_end_premise = dataset_info["end_premise"]

  if(has_sep and has_initial_premise):
    df[0] = df[0].map(handle_initial_sep)

  if(has_sep and has_end_premise):
    if(has_sep != True):
      df[0] = df[0].apply(lambda q: handle_end_sep(q, has_sep, ["question:"]))
    else:
      df[0] = df[0].map(handle_end_sep)
  #print(df[0][6])
  return (df[0], df[1])

In [ ]:
for dataset_index in range (0, len(QA_DATASETS)):
  import_dataset(dataset_index)

In [ ]:
def adapt_prompt(prompt_index, question, options):
  return QA_PROMPTS[prompt_index].format(QUESTION = question, OPTIONS = options)

In [ ]:
def preprocess(premises):
  premises.tolist()
  question = []
  answer = []

  for premise in premises:
    split = re.split("\([A-H]\)", premise)
    question.append(split.pop(0))
    answer.append(split)

  return question, answer

## Upload model

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("zero-shot-classification", model="niting3c/llama-2-7b-hf-zero-shot")

In [ ]:
sequences = pipeline(
    "Answer the following QA question printing only the correct option (no need of explanation):\nQUESTION: Tommy glided across the marble floor with ease, but slipped and fell on the wet floor because _____ has more resistance. OPTIONS: [' marble floor ', ' wet floor']\n",
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

In [ ]:
sequences

In [ ]:
sequences[0][0]['generated_text']

In [ ]:
sequences[1][0]['generated_text']

In [ ]:
match = re.search("(Correct answer: \([A-D]\)).*", seq['generated_text'])

In [ ]:
match

In [ ]:
match = re.sub("(Correct answer: \([A-D]\)).", "", match.group(0))

In [ ]:
match.replace('.', '').replace(' ', '')

In [ ]:
sequences = []

In [ ]:
def askLlama2QA(data, solutions, prompt_index = None):
  q, a = preprocess(data)
  results = []
  prompt = []
  solutions = solutions.tolist()

  for question, options, solution in zip(q, a, solutions):
    if(prompt_index != None):
      prompt = adapt_prompt(prompt_index, question, options)
    else:
      prompt = question

    sequence = pipeline(
      prompt,
      do_sample=True,
      top_k=10,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_length=200
    )

    print(sequence[0]['generated_text'], "\n\n\n")

    #print(flan_tokenizer.batch_decode(output_sequences, skip_special_tokens=True)[0].replace(" ", "").replace(".", "").lower())
    #print(solution.replace(" ", "").replace(".", "").lower())

    #print(f"FLAN answer: {result}, correct answer: {solution}")

    #for i in range (0, len(q)):
    #  print("Question ", i+1, ": ", sequences[i][0]['generated_text'])
      #print(re.search("(Correct answer: \([A-D]\)).*", sequences[i][0]['generated_text']))
      #print(re.search("(Correct answer: \([A-D]\)).*", sequences[i][0]['generated_text']), ", ", solution, ": ", re.search("(Correct answer: \([A-D]\)).*", sequences[i][0]['generated_text']).contains(solution))
      #if(re.search("(Correct answer: \([A-D]\)).*", sequences[i][0]['generated_text']).contains(solution)):
      #  results.append(True)
      #else:
      #  results.append(False)


    #results.append(flan_tokenizer.batch_decode(output_sequences, skip_special_tokens=True))

  return results

In [ ]:
def final_tests():
    for dataset_index in range (0, len(QA_DATASETS)):
      for prompt in range (0, 1):
        question, answer = import_dataset(dataset_index)
        results = askLlama2QA(question[0:5], answer[0:5], prompt)

        dataset_name = QA_DATASETS[dataset_index]["name"]

        #corrects = sum(bool(x) for x in results)

        #str = f"Model: {model}\tDataset name: {dataset_name}\tPrompt index: {prompt} - Correct Answers {corrects}/{len(results)} - {corrects/len(results)}"
        #print(str)
        corrects = 0
      print("\n\n")

In [ ]:
final_tests()

In [ ]:
df = pd.read_csv("sciq/sciq_32_100_test.tsv", sep="\t", header=None)

In [ ]:
sequence = pipeline(
      df[0].tolist(),
      do_sample=True,
      top_k=10,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_length=800
    )

In [ ]:
sequence[0][0]['generated_text']

In [ ]:
content = ''
for item in sequence:
  content += item[0]['generated_text'] + "\n\n"

In [ ]:
content = content.encode('ascii', 'ignore').decode('ascii')

In [ ]:
f = open("/content/gdrive/MyDrive/sciq-result", "a")
f.write(content)
f.close()

## TIME COMPUTATION

In [ ]:
import time

In [ ]:
def time_tests():
  for dataset_index in range (0, len(QA_DATASETS)):
      for prompt in range (0, 1):
        question, answer = import_dataset(dataset_index)
        start = time.time()
        results = askLlama2QA(question[0:32], answer[0:32], prompt)
        end = time.time()

        dataset_name = QA_DATASETS[dataset_index]["name"]

        print(f"{dataset_name} time: {end-start}")
      print("\n\n")

In [ ]:
time_tests()

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("quarel/quarel_32_100_test.tsv", sep="\t", header=None)